## About Those Monads

Monads are an algebraic structure that models the notion of sequencing actions in some shared context.

There. That wasn't so bad, was it?

![I think you should be more explicit here in step two.](https://i.pinimg.com/originals/bf/dc/ed/bfdcedf4eebcf6069d61264ea8fcc08c.gif)

### Functional Programming

OK. Let's back up a few steps. When we talk about monads, we're doing it in the context of "functional programming." But what's "functional programming?" If you ask this online, you will literally get snarky answers like "programming with functions!" And this seems fair enough, in some sense. Let's look at an example.

In [1]:
def addFive(i: Int): Int = i + 5

defined function addFive

Yes, this is trivial on purpose.

In [2]:
addFive(7)

res1: Int = 12

In [3]:
addFive(addFive(7))

res2: Int = 17

OK, so we can define and call functions. (It would be a weird language if we couldn't.) "Well," some online will say, "what we really mean is that functions are _first-class_. They're _values_."

In Scala, this adds a wrinkle, but only a minor one: [`def`s aren't functions](https://tpolecat.github.io/2014/06/09/methods-functions.html)! They're methods. But we can convert them to values easily by referring to the argument with a wildcard, like this:

In [4]:
val add5: Int => Int = addFive _

add5: Int => Int = ammonite.$sess.cmd3$Helper$$Lambda$2266/226929317@343b12a5

We see the story told in two ways: first, that we're defining a `val`. Second, that the type of the `val`ue is `Int => Int`. A type with a double-lined arrow in it is a function type.

In [5]:
add5(7)

res4: Int = 12

In [6]:
add5(add5(7))

res5: Int = 17

One of the things that's meant by "functions are first-class" or "functions are values" is that it opens the door to functions taking other functions as arguments, or functions returning functions as results. Functions that take other functions as arguments are called _higher-order_ functions. You're probably familiar with a few already, like `map`:

In [7]:
List(1, 2, 3, 4, 5).map(add5)

res6: List[Int] = List(6, 7, 8, 9, 10)

But there are many more. We've seen above that we can use `addFive` or `add5` twice, just by nesting calls. But there's another way to write this, in fact, in such a way that we can use it with any argument:

In [8]:
val add10 = add5 andThen add5

add10: Int => Int = scala.Function1$$Lambda$312/1346343363@648d8db8

Here I didn't bother declaring the type, since Scala can infer it just fine.

In [9]:
add10(7)

res8: Int = 17

[`andThen`](https://www.scala-lang.org/api/2.12.3/scala/Function1.html#andThen[A](g:R=%3EA):T1=%3EA) is a higher-order _method_: it takes a function as an argument and returns a function that calls the function it's defined on, _and then_ the argument function, as its result. A function or method that takes only other functions as arguments and returns a function as its result is called a _combinator_. Scala supports using methods taking one argument in infix position. It also offers dot- and parenthesis-inference. So alternative ways of writing the above could be:

In [10]:
add5.andThen(add5)

res9: Int => Int = scala.Function1$$Lambda$312/1346343363@417b706f

In [12]:
add5 andThen(add5)

res11: Int => Int = scala.Function1$$Lambda$312/1346343363@67a80522

As trivial as this is, it really is what we mean by "functional programming:" we "build up" programs by "composition," or "combining," functions until we have a whole program.

But we immediately run into problems. For example, what about I/O?

In [13]:
val input = scala.io.StdIn.readLine()

 foo


input: String = "foo"

Well, _that_ didn't go the way we wanted. Instead of getting a function we could run later to get input, we got input immediately. Let's take another tack:

In [14]:
val input = () => scala.io.StdIn.readLine()

input: () => String = ammonite.$sess.cmd13$Helper$$Lambda$2576/190719038@5470afcf

In [15]:
input()

 foo


res14: String = "foo"

Now we have a function that takes input from the user, at the cost of artificially taking the standard meaningless value (`()`) as an argument. Why? Isn't `scala.io.StdIn.readLine` a perfectly good function?

Well, no, it isn't, precisely because it "does stuff" the moment we try to use it. But mathematically speaking, `input()` isn't a function either!

In [16]:
input()

 foo


res15: String = "foo"

In [17]:
input()

 bar


res16: String = "bar"

In [18]:
input()

 baz


res17: String = "baz"

Mathematically, a _function_ maps _each_ possible argument to _one_ possible result value. Here, we see the only possible argument, _nothing_, yield completely different results: `"foo"`, `"bar"`, and `"baz"`. `input()` is not a function. Its result is _dependent upon some context_ (that we know nothing about).

So what? Why do we care about how things are defined "mathematically?" I'm imposing an artificial constraint in insisting on using `andThen`, aren't I? This isn't complicated:

In [2]:
println("What's your name?"); val name = scala.io.StdIn.readLine(); println(s"Hello, $name!")

What's your name?


 Paul


Hello, Paul!


name: String = "Paul"

There's something to be said for this. It's familiar to most programmers, even those who don't know Scala. Things happen in order. The semicolon separates and sequences things. On the other hand, There's something a little weird, isn't there, about those free-floating lines, "What's your nane?", " Paul", and "Hello, Paul!", without a "[2]:" or whatever in front of it, and then the "name: String" result _after_ the "Hello, Paul!" This is just a variation on the theme of trying to use `scala.io.StdIn.readLine()` as a function by "assigning it to a `val`" to try to use with `andThen`. The value of the expression depends on a context that no longer exists, and the code has "done stuff" as soon as I entered it, including producing output, _before returning the result_.